In [6]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

In [7]:
data = pd.read_csv('scdb-test-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [8]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

56

In [9]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

buyoffer_count = 0
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'INVERSE_TXN':
        buyoffer_count += 1   


print(f"#INVERSE_TXNs =  {buyoffer_count}")

#BUYOFFERs =  8


In [10]:
time_all_committed_rfqs = {}
time_passed_rfqs = []
validated_time = []
accepted_time = []
committed_time = []

buyoffer_underfit = []
buyoffer_underfit_dict = {}

buyoffer_overfit = []
buyoffer_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "INVERSE_TXN" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            buyoffer_underfit.append(temp_list)
            buyoffer_underfit_dict[tx_id(i)] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            buyoffer_overfit.append(temp_list)
            buyoffer_overfit_dict[tx_id[i]] = temp_list

TypeError: 'list' object is not callable

In [ ]:
# Analysis of RFQ Transaction
if len(time_passed_rfqs) < 2:
    time_rfqs.append(time_passed_rfqs[0])
rfq_mean = "{:.2f}".format(stat.mean(time_passed_rfqs))
rfq_median = stat.median(time_passed_rfqs)
rfq_max = max(time_passed_rfqs)
rfq_min = min(time_passed_rfqs)
rfq_std_dev = "{:.2f}".format(stat.mean(time_passed_rfqs))

print(rfq_mean)
print(rfq_median)
print(rfq_max)
print(rfq_min)
print(rfq_std_dev)
print(f"total INVERSE_TXNs = {len(time_passed_rfqs)}")

rfq_dict = {
               'min': rfq_min,
               'max': rfq_max,
               'average': rfq_mean,
                'median' : rfq_median,
               'standard_deviation' : rfq_std_dev,
               '# of txs' : len(time_passed_rfqs)}

169.50
131.0
316
100
169.50
total RFQs = 4


In [ ]:
len(buyoffer_overfit_dict)

1

In [ ]:
buyoffer_overfit_dict

{'3aa1bfc78754b9444c67e865b7aa2a15ee58d2e722770743f65720330596e0b7': [24,
  44,
  51,
  208,
  277,
  279,
  340]}

In [ ]:
results = pd.DataFrame(rfq_dict, index = ['INVERSE_TXN'])
results.to_csv('inversetxn_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
RFQ,100,316,169.50,131.0,169.50,4


In [ ]:
results.to_csv('inversetxn_analysis.csv', sep=',', encoding='utf-8')

In [ ]:
results

,min,max,average,median,standard_deviation,# of txs
RFQ,100,316,169.50,131.0,169.50,4
